## Prerequisites

1. divvy
1. anaconda python
1. aws account
1. awscli
1. installation: `conda --yes install s3fs pyarrow fastparquet tqdm seaborn holoviews`

## trick list

1. fullscreen everything
1. command-tab
1. divvy - command-shift-D
1. conda set yes always true `conda config --set always_yes True`
1. `! aws s3 ls` --> `pd.read_txt`
1. write & submit reflow batch
1. itertools.product --> pandas dataframe
1. itertools.chain
1. format strings
1. tqdm
1. `df.apply(lambda row: {id}_{abundance}_k{ksize}.format(**row), axis=1)`
1. git branch
1. grep
1. awk
1. sed
1. wc -l
1. make tidy df, chain pandas commands: `df.reset_index().stack().reset_index()`
1. save as parquet
1. read lots of csv files and concat, save as parquet
1. screen/tmux
1. read csvs from s3 (s3fs)
1. seaborn facetgrid
1. hover-able stuff with holoviews
1. aliases (ll, exa)
1. soft links (`mv ~/anaconda /mnt/data/anaconda && sudo ln -s /mnt/data/anaconda ~/anaconda`)
1. tunnel jupyter notebook
1. `watch --differences -n 60 check_reflow_status.py`
1. `tail -n 20 log* | less -S`
1. xargs
1. `grep canceled listbatch.txt |cut -f 1 -d ' ' | sed 's/^/log./g' | xargs tail -n 20 - | less`
1. markdown cells in jupyter notebook
1. `pd.options.display.max_colwidth = 500`
1. kwargs=dict(), `**kwargs`
1. 
```
✘  Fri 21 Sep - 01:29  ~ 
  mv ~/anaconda /mnt/data/anaconda

 Fri 21 Sep - 01:31  ~ 
  ln -s /mnt/data/anaconda ~/anaconda
```
1. regex, regex101, pandas extract
1. 
```
%load_ext autoreload
%autoreload 2
```
1. write python file in jupyter, import it
1. groupby
1. %debug
1. join
1. colorbrewer
1. download fastqs from SRA with reflow
1. `diff <(gunzip -c tick_1_S1_first1Mreads_R1_adapterremoval_trimmed.fastq.gz) <(gunzip -c tick_1_S1_R1_001_first1Mreads_fastp_trimmed.fastq.gz)`
1. pbpaste/pbcopy
1. `gist --private` command line 
1. jupter notebook keystrokes: 
    - Ctrl-M-A add cell above
    - Ctrl-M-B add cell below
    - Ctrl-M-d d delete cell
    - Ctrl-M-i interrupt
    - Ctrl-M-0 restart

In [1]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

                           PRE tabula-muris-k21-dna/
                           PRE tabula-muris-k21-protein/
                           PRE tabula-muris-k27-dna/
                           PRE tabula-muris-k27-protein/
                           PRE tabula-muris-k33-dna/
                           PRE tabula-muris-k33-protein/
                           PRE tabula-muris-k51-dna/
                           PRE tabula-muris-k51-protein/


In [ ]:
! aws s3 ls s3://olgabot-maca/facs/sourmash_compare/ > aws_s3_ls_tabula_muris_compare.txt

aws_s3_ls = pd.read_table('aws_s3_ls_tabula_muris_compare.txt', 
                          delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
print(aws_s3_ls.shape)
aws_s3_ls.head()

In [ ]:
databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
databases['database_name'] = databases['database_name'].str.strip('/')
databases = databases.drop('is_prefix', axis=1)
databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
databases['database'] = databases['database_name'].map(lambda x: f'{prefix}/{x}/{x}/')
databases = databases.set_index('database_name')
databases